# Imports 

In [ ]:
#from dataloaders import Dataloader as DL

import os
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
import arpespythontools as arp
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, interactive_output
import astropy 
from mpl_toolkits.mplot3d import Axes3D
import pickle
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from datetime import datetime
import csv
from astropy.convolution import convolve, Box2DKernel
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
# Run the notebook script
%run data_loader.ipynb

# Access the cached data directly by variable names
map_data2 = map_data2
phi = phi
energy = energy
theta = theta
theta_v = theta_v


In [ ]:
def angle_to_wavevector_binding(E_pho, work_f, theta, phi):
    #CONVERTS PIXELS TO THETA
    global theta_v
    
    #print(theta_v)
    E_k = E_pho - work_f
    k = 0.512*np.sqrt(E_k)
    theta_rad = np.radians(theta_v)
    phi_rad = np.radians(phi)
    
    #K_x and K_Y 
    k_x = k*np.sin(theta_rad)
    k_y = k*np.sin(phi_rad)
    
    #k_x = np.degrees(k_x)
    #k_y = np.degrees(k_y)
    return k_x, k_y
        
    
global k_x
global k_y

k_x, k_y = angle_to_wavevector_binding(85, 4, theta_v, phi)

# Laplacian Taken in k_x

In [ ]:
def plt_plot_laplacian(k_y1, k_y2, k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, clim_min=-1, clim_max=1):
    surface1 = arp.plane_slice(map_data2.transpose([2, 0, 1]), k_y, k_y1, k_y2)

    diff2 = laplacian(surface1)

    energy_mask = (energy >= Ev_Lowerbound) & (energy <= Ev_Upperbound)
    diff2 = diff2[energy_mask]
    energy_zoomed = energy[energy_mask]

    k_x_mask = (k_x >= k_x1) & (k_x <= k_x2)
    diff2 = diff2[:, k_x_mask]
    k_x_zoomed = k_x[k_x_mask]

    plt.figure(figsize=(8, 6))
    plt.imshow(diff2, aspect="auto", origin='lower',
               extent=[k_x_zoomed[0], k_x_zoomed[-1], energy_zoomed[0], energy_zoomed[-1]],
               cmap='terrain_r', vmin=clim_min, vmax=clim_max)
    plt.xlabel("Wavevector (k_x)")
    plt.ylabel("Energy (eV)")
    plt.colorbar(label="Laplacian Intensity")
    plt.title("Laplacian of ARPES Data (k_x slice)")
    plt.show()
    
k_y1_slider = widgets.FloatSlider(value=k_y[0], min=np.min(k_y), max=np.max(k_y), step=0.01, description='k_y1')
k_y2_slider = widgets.FloatSlider(value=k_y[-1], min=np.min(k_y), max=np.max(k_y), step=0.01, description='k_y2')
k_x1_slider = widgets.FloatSlider(value=k_x[0], min=np.min(k_x), max=np.max(k_x), step=0.01, description='k_x1')
k_x2_slider = widgets.FloatSlider(value=k_x[-1], min=np.min(k_x), max=np.max(k_x), step=0.01, description='k_x2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=energy[0], min=energy[0], max=energy[-1], step=0.01, description='E min')
Ev_Upperbound_slider = widgets.FloatSlider(value=energy[-1], min=energy[0], max=energy[-1], step=0.01, description='E max')
clim_min_slider = widgets.FloatSlider(value=-10000, min=-10000, max=0, step=0.1, description="Color Min")
clim_max_slider = widgets.FloatSlider(value=800, min=0, max=10000, step=0.1, description="Color Max")

widgets.interact(
    plt_plot_laplacian,
    k_y1=k_y1_slider,
    k_y2=k_y2_slider,
    k_x1=k_x1_slider,
    k_x2=k_x2_slider,
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider,
    clim_min=clim_min_slider,
    clim_max=clim_max_slider,
)



# Laplacian Taken in k_y

In [ ]:
def plt_plot_laplacian(k_y1, k_y2, k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, clim_min=-1, clim_max=1):
    surface1 = arp.plane_slice(map_data2.transpose([1, 0, 2]), k_x, k_x1, k_x2)

    diff2 = laplacian(surface1)

    energy_mask = (energy >= Ev_Lowerbound) & (energy <= Ev_Upperbound)
    diff2 = diff2[energy_mask]
    energy_zoomed = energy[energy_mask]

    k_y_mask = (k_y >= k_y1) & (k_y <= k_y2)
    diff2 = diff2[:, k_y_mask]
    k_y_zoomed = k_y[k_y_mask]

    plt.figure(figsize=(8, 6))
    plt.imshow(diff2, aspect="auto", origin='lower',
               extent=[k_y_zoomed[0], k_y_zoomed[-1], energy_zoomed[0], energy_zoomed[-1]],
               cmap='terrain_r', vmin=clim_min, vmax=clim_max)
    plt.xlabel("Wavevector (k_y)")
    plt.ylabel("Energy (eV)")
    plt.colorbar(label="Laplacian Intensity")
    plt.title("Laplacian of ARPES Data (k_y slice)")
    plt.show()
    
k_y1_slider = widgets.FloatSlider(value=k_y[0], min=np.min(k_y), max=np.max(k_y), step=0.01, description='k_y1')
k_y2_slider = widgets.FloatSlider(value=k_y[-1], min=np.min(k_y), max=np.max(k_y), step=0.01, description='k_y2')
k_x1_slider = widgets.FloatSlider(value=k_x[0], min=np.min(k_x), max=np.max(k_x), step=0.01, description='k_x1')
k_x2_slider = widgets.FloatSlider(value=k_x[-1], min=np.min(k_x), max=np.max(k_x), step=0.01, description='k_x2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=energy[0], min=energy[0], max=energy[-1], step=0.01, description='E min')
Ev_Upperbound_slider = widgets.FloatSlider(value=energy[-1], min=energy[0], max=energy[-1], step=0.01, description='E max')
clim_min_slider = widgets.FloatSlider(value= -372903, min=-1000000, max=0, step=0.1, description="Color Min")
clim_max_slider = widgets.FloatSlider(value=800, min=0, max=10000, step=0.1, description="Color Max")

widgets.interact(
    plt_plot_laplacian,
    k_y1=k_y1_slider,
    k_y2=k_y2_slider,
    k_x1=k_x1_slider,
    k_x2=k_x2_slider,
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider,
    clim_min=clim_min_slider,
    clim_max=clim_max_slider,
)